In [1]:

import re
import openpyxl
import pandas as pd
import os

# Data import 

In [52]:
# import date 
df = pd.read_excel('D:\\aphelio\\Take home - exercice 1 - product name and version parsing.xlsx', sheet_name='Dataset')

# Data Exploration

In [53]:
# Look at the first few rows of the dataframe to get an idea of the data
df.head()

,vendor,product,version,update,versionStartIncluding,versionStartExcluding,versionEndIncluding,versionEndExcluding
0,hikvision,ds-2cd2021g1-i\(w\),-,*,NaN,NaN,NaN,NaN
1,hikvision,ds-2cd2023g2-i\(u\),-,*,NaN,NaN,NaN,NaN
2,hikvision,ds-2cd2026g2-iu\/sl,-,*,NaN,NaN,NaN,NaN
3,hikvision,ds-2cd2027g2-l\(u\),-,*,NaN,NaN,NaN,NaN
4,hikvision,ds-2cd2027g2-lu\/sl,-,*,NaN,NaN,NaN,NaN


In [54]:
# Look at unique product names to understand the naming inconsistencies
df['product'].unique()

array(['ds-2cd2021g1-i\\(w\\)', 'ds-2cd2023g2-i\\(u\\)',
       'ds-2cd2026g2-iu\\/sl', 'ds-2cd2027g2-l\\(u\\)',
       'ds-2cd2027g2-lu\\/sl', 'ds-2cd2032-i', 'ds-2cd2043g2-i\\(u\\)',
       'ds-2cd2046g2-iu\\/sl', 'ds-2cd2047g2-l\\(u\\)',
       'ds-2cd2063g2-i\\(u\\)', 'ds-2cd2066g2-i\\(u\\)',
       'ds-2cd2066g2-iu\\/sl', 'ds-2cd2083g2-i\\(u\\)',
       'ds-2cd2086g2-i\\(u\\)', 'ds-2cd2086g2-iu\\/sl',
       'ds-2cd2087g2-l\\(u\\)', 'ds-2cd2112-i',
       'ds-2cd2121g0-i\\(w\\)\\(s\\)', 'ds-2cd2121g1-i\\(w\\)',
       'ds-2cd2121g1-idw', 'ds-2cd2121g1', 'ds-2cd2123g2-i\\(s\\)',
       'ds-2cd2123g2-iu', 'ds-2cd2127g2-\\(-su\\)', 'ds-2cd2132-i',
       'ds-2cd2143g2-i\\(s\\)', 'ds-2cd2143g2-iu',
       'ds-2cd2147g2-l\\(su\\)', 'ds-2cd2163g2-i\\(s\\)',
       'ds-2cd2163g2-iu', 'ds-2cd2166g2-i\\(su\\)',
       'ds-2cd2183g2-i\\(s\\)', 'ds-2cd2183g2-i\\(u\\)',
       'ds-2cd2183g2-iu', 'ds-2cd2186g2-i\\(su\\)', 'ds-2cd2186g2-isu',
       'ds-2cd2212-i5', 'ds-2cd2232-i5', 'ds-2cd2312

In [55]:
len(df['product'].unique())

763

In [56]:
# Look at unique version names to understand the naming inconsistencies
df['version'].unique()

array(['-', '*', '4.1.0_b130111', '4.0.1', '2.2.10'], dtype=object)

In [57]:
len(df['version'].unique())

5

# Data cleaning
## Removing duplicated records

In [58]:
# check if the dataframe contains duplicate values
duplicates = df.duplicated()

# print the number of duplicate rows
print(f'Number of duplicate rows = {duplicates.sum()}')

Number of duplicate rows = 1


yes, the dataframe contains duplicated lines we must delete them

In [59]:
#see 
print(df[df.duplicated()])

        vendor       product version update versionStartIncluding  \
384  hikvision  ds-2cd7153-e       -      *                   NaN   

     versionStartExcluding versionEndIncluding versionEndExcluding  
384                    NaN                 NaN                 NaN  


In [60]:
len(df)

796

the dataset contains 796 records

In [61]:
# Remove duplicate rows
df = df.drop_duplicates()

In [62]:
len(df)

795

## Identifying the inconsistencies in product name

We use this pdf to understand naming rules : 
https://mk-smart-solutions.de/media/image/storage/opc/naming%20rule%2020210118.pdf

Some examples are available in this website :  https://www.hikvision.com/europe/search/?q=ds&active=Products





In the dataset provided, there are several inconsistencies present in the product names that need to be addressed. For example, the product name 'hikvision ds-2cd2183g2-iu' does not include parentheses around 'u', which is a standard we'd like to establish. Additionally, in the product name 'hikvision ds-2cd2186g2-i(su)', not only is 'su' enclosed within parentheses, but the parentheses are also escaped, which adds an unnecessary layer of complexity. Lastly, in 'hikvision ds-2cd2026g2-iu/sl', 'sl' appears after a '\/', which is inconsistent with our other product naming conventions. To standardize our dataset, these anomalies need to be addressed


In [2]:
# Define function to standardize product names

def standardize_product_name(product_name):
    
    # Convert to lower case
    product_name = product_name.lower()
    
    # Add parentheses around 'u' if it follows 'i' and is at the end, and isn't already in parentheses
    product_name = re.sub(r"(i)u$", r"\1(u)", product_name)

    # Remove escaping from parentheses
    product_name = product_name.replace(r"\(", "(").replace(r"\)", ")")

    # replacing'\/' by 
    product_name = product_name.replace(r"\/", "/")
    
    
    return product_name

In [116]:
standardize_product_name('ds-2cd2183g2-iu')


'ds-2cd2183g2-i(u)'

In [117]:
standardize_product_name('ds-2cd3026g2-iu\/sl')

'ds-2cd3026g2-iu/sl'

In [118]:
standardize_product_name('ds-2cd3123g2-i\(s\)u')

'ds-2cd3123g2-i(s)u'

## Identifying the inconsistencies in version and  update 

In the provided data sample, we can observe some inconsistencies regarding the representation of product names, version numbers, and updates. These inconsistencies could cause issues when analyzing or comparing the data.

the build number is embedded within the version column for ds-2cd7153-e_firmware, also from hikvision. The version is listed as 4.1.0_b130111, where b130111 appears to be the build number but is included within the version.

The second entry highlights another variation in how the build number is represented. In this case, for the product dvr_ds-7204_firmware from hikvision, the build number is prefixed with build_, making it build_131009 instead of just 131009.

These inconsistent representations can complicate the data cleaning and standardization process, but by identifying and categorizing these problems, we can develop effective strategies to address them.

In [3]:
# Define function to standardize version numbers and update details
def standardize_version_and_update(version, update):
    # Check if version has an embedded build number
    if "_b" in version:
        # If yes, split the version into version number and build number
        version, build = version.split("_b")
        # put the build number into the update details
        update = build
        
    if "build_" in update:
        # If yes, remove the "build_" prefix from the update
        update = re.sub(r"build_", "", update)    
        
    
    return version, update

In [4]:
standardize_version_and_update('4.1.0_b130111','*')

('4.1.0', '130111')

In [5]:
standardize_version_and_update('4.1.0','*')

('4.1.0', '*')

In [8]:
standardize_version_and_update('2.2.1','build_131009')

('2.2.1', '131009')

In [9]:
standardize_version_and_update('-','*')

('-', '*')

## Standarazing a batch of records

Pandas DataFrame's **apply** funvction is used in the standardization functions, these are already quite capable of handling batch operations. This is because the apply function operates on whole series of data rather than individual elements, leading to efficiencies in execution.

However, if the  dataset is very large, we are  concerned about memory usage, we might want to process the data in **chunks**. we can use Pandas' read_excel function with the **chunksize** parameter to read and process the data in chunks.


Note that processing the data in chunks like this will be slower than processing it all at once, because the function application (apply) has to be performed separately for each chunk. However, this approach enables us to work with datasets that are too large to fit into memory, which might be necessary in some situations.

Finnaly, we write each processed chunk to a new Excel or CSV file. This way, the memory is freed up as each chunk is written. 


In [118]:
import openpyxl
import pandas as pd
import os

def process_chunk(chunk, output_filepath,row_num, mode):
    
    print(chunk.columns)  # Add this line to debug
    print(chunk)

    # Apply the function to the 'Product' column
    chunk['product'] = chunk['product'].apply(standardize_product_name)
        
    # Apply the function to the 'Version' and 'Update' columns
    chunk['version'], chunk['update'] = zip(*chunk.apply(lambda row: standardize_version_and_update(row['version'], row['update']), axis=1))
        
    # Write the processed chunk to Excel
    writer = pd.ExcelWriter(output_filepath, engine='openpyxl', mode=mode)
    header = True
    if mode == 'a' :
        writer = pd.ExcelWriter(output_filepath, engine='openpyxl', mode='a', if_sheet_exists='overlay') 
        header =None
    with writer:
        chunk.to_excel(writer,  startrow=row_num,index=False,header=header)
        
 

# Batch processing function

def batch_parse(filepath, sheet_number, output_filepath, chunksize=10000):
    wb = openpyxl.load_workbook(filename = filepath, read_only=True)
    ws = wb[wb.sheetnames[sheet_number]]
    header = [cell.value for cell in ws[1]]  # read the header in the first row
    data = []
    chunk_number =0
    for i, row in enumerate(ws.rows):
        data.append([cell.value for cell in row])
        if (i+1) % chunksize == 0 or (i+1) == ws.max_row:
            chunk_number +=1
            mode = 'a' if os.path.exists(output_filepath) else 'w'
            if (mode=='a'):
                chunk = pd.DataFrame(data, columns=header)  # use the header for each chunk
            else :
                chunk = pd.DataFrame(data[1:], columns=header)  # use the header for each chunk
            
            process_chunk(chunk, output_filepath,(chunk_number-1)*chunksize, mode)
            data = []
    print(f"Processed data written to: {output_filepath}")



In [119]:
batch_parse('D:\\aphelio\\Take home - exercice 1 - product name and version parsing.xlsx',1,'D:\\aphelio\\cleaned_dataset.xlsx',100)

Index(['vendor', 'product', 'version', 'update', 'versionStartIncluding',
       'versionStartExcluding', 'versionEndIncluding', 'versionEndExcluding'],
      dtype='object')
       vendor              product version update versionStartIncluding  \
0   hikvision  ds-2cd2021g1-i\(w\)       -      *                         
1   hikvision  ds-2cd2023g2-i\(u\)       -      *                         
2   hikvision  ds-2cd2026g2-iu\/sl       -      *                         
3   hikvision  ds-2cd2027g2-l\(u\)       -      *                         
4   hikvision  ds-2cd2027g2-lu\/sl       -      *                         
..        ...                  ...     ...    ...                   ...   
94  hikvision     ds-2cd2743g2-izs       -      *                         
95  hikvision     ds-2cd2763g2-izs       -      *                         
96  hikvision     ds-2cd2766g2-izs       -      *                         
97  hikvision     ds-2cd2783g2-izs       -      *                         


Index(['vendor', 'product', 'version', 'update', 'versionStartIncluding',
       'versionStartExcluding', 'versionEndIncluding', 'versionEndExcluding'],
      dtype='object')
       vendor                            product version update  \
0   hikvision        ds-2df8236i5x-aelw_firmware       -      *   
1   hikvision   ds-2df8242i5x-ael\(t3\)_firmware       -      *   
2   hikvision  ds-2df8242i5x-aelw\(t3\)_firmware       -      *   
3   hikvision  ds-2df8242i5x-aelw\(t5\)_firmware       -      *   
4   hikvision    ds-2df8242ix-ael\(t5\)_firmware       -      *   
..        ...                                ...     ...    ...   
95  hikvision         ds-7616ni-q2\/16p_firmware       *      *   
96  hikvision              ds-7616ni-q2_firmware       *      *   
97  hikvision           ds-76xxx_series_firmware       *      *   
98  hikvision           ds-77xxx_series_firmware       *      *   
99  hikvision                 ds-a71024_firmware       *      *   

   versionStartInclu

# Data Standardization Report

## Introduction

We were provided with a dataset containing product names, version numbers, and updates from a specific manufacturer. During the preliminary exploration, we identified inconsistencies in these fields that could hinder data analysis and comparison.

## Approach

We used Python for the data cleaning process and applied the `pandas` library's powerful data manipulation capabilities. To ensure scalability and manage memory usage, we processed the data in batches, taking advantage of pandas' `read_excel` function's `chunksize` parameter.

The cleaning process comprised of the following steps:

1. **Importing Data**: We imported the data from an Excel file into a pandas DataFrame for further processing.

2. **Removing Duplicates**: We identified and removed duplicate entries in the dataset to ensure the uniqueness of records.

3. **Identifying Inconsistencies**: We analyzed unique product names and versions to understand the naming inconsistencies. 

4. **Standardizing Product Names**: We created a function to standardize the product names based on the observed naming rules. The function converted product names to lowercase, added parentheses around 'u' if it follows 'i', removed escaping from parentheses, and replaced '\/' with '/'.

5. **Standardizing Version and Update Details**: We observed inconsistent representations of build numbers in version and update details. We created a function to handle these inconsistencies. If the version had an embedded build number, it was split into version number and build number, and the build number was added to the update details. If the update had the prefix 'build_', it was removed.

6. **Batch Processing**: We processed the data in chunks to handle large datasets that might not fit into memory. We applied the standardization functions to each chunk and wrote the processed chunk to a new Excel file.

## Findings

Our data cleaning process successfully standardized the product names, version numbers, and updates, thus resolving the initial inconsistencies that were present in the dataset. 

## Recommendations for Future Work (Bonus part)

For maintaining this standardization in the future and mitigating the introduction of new inconsistencies, we recommend the following strategies:

1. **Data Entry Guidelines**: Develop clear guidelines for entering data into the system. This could involve specific instructions on naming conventions for products and versions. We used https://mk-smart-solutions.de/media/image/storage/opc/naming%20rule%2020210118.pdf to understand naming rules (Refer to the detailed description provided within the document).

2. **Data Validation**: Implement data validation mechanisms at the data entry level. This could involve checks for naming conventions and restrictions on the type of characters allowed.

3. **Periodic Auditing and Cleaning**: Conduct regular audits of the data to identify any new inconsistencies. Apply the data cleaning process to new data on a regular basis.

4. **Automated Standardization Scripts**: Continue using and updating the data cleaning script to standardize new data. This can be automated to run at regular intervals.

5. **Training**: Ensure that all individuals involved in data entry are trained on the data entry guidelines and understand the importance of maintaining consistency in data entry.

These recommendations aim to ensure the consistency and reliability of data over time, facilitating smooth data analysis and comparison.


## Bonus part (strategy)

A regular expression (regex) is a powerful tool that is essentially a sequence of characters defining a search pattern. This pattern can be used in string searching algorithms, find-and-replace operations, input validation, data parsing and much more.

Given a certain naming rule for a product (like the camera names in your examples), we can generate a regex pattern that exactly matches this rule. This process involves interpreting each part of the naming rule and translating it into regex syntax. Here's how this works for the camera naming rule:

Basic structure: The basic structure of the camera name consists of various parts separated by spaces or specific characters (like '-', '_', etc.). In regex, we can directly use these parts. For example, 'DS-2' at the start of the name can be represented as ^DS-2 in regex, where ^ denotes the start of the string.

Options: If a part can have multiple options, we use [] or () to represent these in regex. For example, if a part can be either '3', '5', '7', or '9', we can represent this as [3579]. If a part can be either 'A', 'D', or 'ids-2', we represent it as (?:A|D|ids-2).

Optional parts: If a part is optional (it can be present or not), we append a ? after its representation in regex. For example, if 'I3', 'I5', 'I8', or 'IA' are optional, we represent it as (?:I3|I5|I8|IA)?.

End of string: We denote the end of the string with $ in regex. This ensures that the string strictly follows our defined pattern until its end.

By combining these regex rules, we generate a pattern that can verify if a camera name strictly follows the given naming rule. A Python function can then use this regex pattern with the re.match() function to check if a given string (camera name) matches this pattern, thus ensuring it adheres to the naming rule.